In [5]:
import xarray as xr
from pathlib import Path
from ipwgml.data import list_files
from ipwgml.config import get_data_path

data_path = Path(r"F:\Academics\Research_paper\Programs\Data\spr\gmi\evaluation\gridded\target")

nc_files = list(data_path.glob("*.nc"))

if not nc_files:
    print("No .nc files found in the specified directory.")
else:
   
    dataset = xr.open_dataset(nc_files[0])
    ipwgml_path = get_data_path()
    xr.load_dataset(ipwgml_path / nc_files[0])

dataset


<xarray.Dataset> Size: 334MB
Dimensions:              (samples: 4636128)
Dimensions without coordinates: samples
Data variables:
    time                 (samples) datetime64[ns] 37MB ...
    surface_precip       (samples) float64 37MB ...
    radar_quality_index  (samples) float64 37MB ...
    valid_fraction       (samples) float64 37MB ...
    precip_fraction      (samples) float64 37MB ...
    snow_fraction        (samples) float64 37MB ...
    hail_fraction        (samples) float64 37MB ...
    convective_fraction  (samples) float64 37MB ...
    stratiform_fraction  (samples) float64 37MB ...

In [38]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from typing import Tuple

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from ipwgml.target import TargetConfig
from ipwgml.evaluation import Evaluator

target_config = TargetConfig(min_rqi=0.5)
evaluator = Evaluator(
    reference_sensor="gmi",
    geometry="gridded",
    retrieval_input=[],
    download=False,
    target_config=target_config
)

In [40]:
import numpy as np
from typing import Tuple
import pandas as pd
import xarray as xr
from pathlib import Path
len(evaluator)

207

In [41]:
IMERG_PATH= Path(r'F:\Academics\Research_paper\Programs\Data\spr\gmi\training\gridded')
for path in IMERG_PATH.glob("**/*.nc"):
    print(path.name)  # Print the filename to inspect its structure


geo_20191231233255.nc
geo_20191231233417.nc
geo_20191231233538.nc
geo_20191231233634.nc
geo_20200102001320.nc
geo_20200102001424.nc
geo_20200102001531.nc
geo_20200102001658.nc
geo_20200102001828.nc
geo_20200102001954.nc
geo_20200102014705.nc
geo_20200102014750.nc
geo_20200102014832.nc
geo_20200102014939.nc
geo_20200102015032.nc
geo_20200102015054.nc
geo_20200102015200.nc
geo_20200102015250.nc
geo_20200102032252.nc
geo_20200102032437.nc
geo_20200102032554.nc
geo_20200102032643.nc
geo_20200102081951.nc
geo_20200102082100.nc
geo_20200102082137.nc
geo_20200102082309.nc
geo_20200102095124.nc
geo_20200102095221.nc
geo_20200102095324.nc
geo_20200102095409.nc
geo_20200102095449.nc
geo_20200102095537.nc
geo_20200102095645.nc
geo_20200102095651.nc
geo_20200102095821.nc
geo_20200102100034.nc
geo_20200102112600.nc
geo_20200102112638.nc
geo_20200102112719.nc
geo_20200102112756.nc
geo_20200102112856.nc
geo_20200102112958.nc
geo_20200102113058.nc
geo_20200102113132.nc
geo_20200102232151.nc
geo_202001

In [42]:
import re
from pathlib import Path
import numpy as np
import pandas as pd

def filename_to_date(path: Path) -> np.datetime64:
    """
    Determine the start time of an IMERG file.

    Args:
        path: A Path object pointing to an IMERG file.

    Return:
        A numpy.datetime64 object representing the start time of the IMERG file.
    """
    path = Path(path)
    # Check if the filename starts with 'target_'
    if path.stem.startswith("gmi_"):
        # Extract the filename without the extension
        time_str = path.stem[7:]  # 'target_' is 7 characters long, so we start from index 7
        
        # Validate the time_str length to ensure it has the correct format
        if len(time_str) == 14 and re.match(r"^\d{14}$", time_str):
            try:
                # Convert the extracted string into a datetime64 object
                time = pd.to_datetime(time_str, format="%Y%m%d%H%M%S")
                return time.to_datetime64()
            except Exception as e:
                print(f"Error parsing {path.name}: {e}")
                raise
        else:
            raise ValueError(f"Filename does not match expected format: {path.name}")
    else:
        # Log files that don't start with 'target_'
        print(f"Filename does not start with 'target_': {path.name}")
        return None  # Skip this file

# Update the code to handle and filter valid files
imerg_files = sorted(list(IMERG_PATH.glob("**/*.nc")))

# Dictionary comprehension with filtering to avoid invalid filenames
imerg_files_dict = {}
for path in imerg_files:
    try:
        date = filename_to_date(path)
        if date is not None:  # Only add valid files
            imerg_files_dict[date] = path
    except ValueError as e:
        print(e)


Filename does not start with 'target_': geo_20191231233255.nc
Filename does not start with 'target_': geo_20191231233417.nc
Filename does not start with 'target_': geo_20191231233538.nc
Filename does not start with 'target_': geo_20191231233634.nc
Filename does not start with 'target_': geo_20200102001320.nc
Filename does not start with 'target_': geo_20200102001424.nc
Filename does not start with 'target_': geo_20200102001531.nc
Filename does not start with 'target_': geo_20200102001658.nc
Filename does not start with 'target_': geo_20200102001828.nc
Filename does not start with 'target_': geo_20200102001954.nc
Filename does not start with 'target_': geo_20200102014705.nc
Filename does not start with 'target_': geo_20200102014750.nc
Filename does not start with 'target_': geo_20200102014832.nc
Filename does not start with 'target_': geo_20200102014939.nc
Filename does not start with 'target_': geo_20200102015032.nc
Filename does not start with 'target_': geo_20200102015054.nc
Filename

In [47]:
def load_imerg_precip(path: Path, bounds: Tuple[float, float, float, float]) -> xr.Dataset:
    """
    Load IMERG precipitation.

    Args:
        path: A Path object pointing to the IMERG file to load.
        bounds: A tuple ``(lon_min, lat_min, lon_max, lat_max)`` defining a bounding box for the data to load.

    Return:
    """
    lon_min, lat_min, lon_max, lat_max = bounds
    with h5py.File(path) as data:
        lons = data["Grid/lon"][:]
        lats = data["Grid/lat"][:]
        lon_mask = np.where((lon_min <= lons) * (lons <= lon_max))[0]
        lon_start, lon_end = lon_mask.min(), lon_mask.max()
        lon_slc = slice(lon_start, lon_end)
        
        lat_mask = np.where((lat_min <= lats) * (lats <= lat_max))[0]
        lat_start, lat_end = lat_mask.min(), lat_mask.max()
        lat_slc = slice(lat_start, lat_end)
        
        surface_precip = data["Grid/precipitation"][0, lon_slc, lat_slc]
        precip_flag = surface_precip > 1e-2
        heavy_precip_flag = surface_precip > 10.0
        lons = lons[lon_start:lon_end]
        lats = lats[lat_start:lat_end]
        time = np.datetime64("1980-01-06") + data["Grid/time"][:].astype("timedelta64[s]")
        
    return xr.Dataset({
        "longitude": (("longitude"), lons),
        "latitude": (("latitude"), lats),
        "surface_precip": (("longitude", "latitude"), surface_precip),
        "precip_flag": (("longitude", "latitude"), precip_flag.astype(np.int8)),
        "heavy_precip_flag": (("longitude", "latitude"), precip_flag.astype(np.int8)),
        "time": time.astype("datetime64[ns]")
    })

In [48]:
def retrieve_imerg(input_data: xr.Dataset) -> xr.Dataset:
    """
    Retrieval callback function to load IMERG data corresponding to IPWGML SPR evaluation data.

    Args:
        input_data: An xarray.Dataset containing the retrieval input data.
    """
    time_min = np.nanmin(input_data.time.data)
    time_min = time_min.astype("datetime64[h]").astype("datetime64[s]")
    time_max = np.nanmax(input_data.time.data)
    time_max = time_max.astype("datetime64[h]").astype("datetime64[s]") + np.timedelta64(3600, "s")
    lons = input_data.longitude.data
    lats = input_data.latitude.data
    lon_min, lon_max = lons.min(), lons.max()
    lat_min, lat_max = lats.min(), lats.max()
    bounds = (lon_min, lat_min, lon_max, lat_max)
    files = [
        imerg_file for time, imerg_file in imerg_files.items()
        if time_min <= time and time <= time_max
    ]
    imerg_data = xr.concat([load_imerg_precip(path, bounds=bounds) for path in files], "time")
    imerg_data = imerg_data.interp(
        latitude=input_data.latitude,
        longitude=input_data.longitude,
        time=input_data.time,
        method="nearest"
    )
    # Delay conversion to bool since interp ingores bool variables.
    imerg_data["precip_flag"] = imerg_data.precip_flag.astype(bool)
    imerg_data["heavy_precip_flag"] = imerg_data.precip_flag.astype(bool)
    return imerg_data

In [49]:
import xarray as xr

def load_imerg_precip(path: Path, bounds: Tuple[float, float, float, float]) -> xr.Dataset:
    """
    Load IMERG precipitation data from a NetCDF file and subset it based on the provided bounds.

    Args:
        path: A Path object pointing to the NetCDF file.
        bounds: A tuple (lon_min, lat_min, lon_max, lat_max) defining the bounding box for subsetting.

    Return:
        An xarray.Dataset object containing the subsetted IMERG precipitation data.
    """
    # Load the NetCDF file into an xarray Dataset using the 'netcdf4' engine
    ds = xr.open_dataset(path, engine='netcdf4')
    
    # Subset the dataset based on bounds
    lon_min, lat_min, lon_max, lat_max = bounds
    ds_subset = ds.sel(
        lon=slice(lon_min, lon_max),
        lat=slice(lat_min, lat_max)
    )
    
    return ds_subset


In [50]:
fig = evaluator.plot_retrieval_results(scene_index=86, retrieval_fn=retrieve_imerg)

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html